In [2]:
import pandas as pd

In [3]:

LIAR = pd.read_csv("data/LIAR_Dataset.csv")
LIAR.rename(columns={'statement':"title"}, inplace=True)
liar_true_len = len(LIAR.loc[LIAR['label'] == "TRUE"])
liar_false_len = len(LIAR.loc[LIAR['label'] == "FALSE"])
liar_half_true_len = len(LIAR.loc[LIAR['label'] == "half-true"])
print(f"True Claims:{liar_true_len}")
print(f"False Claims:{liar_false_len}")
print(f"Half true Claims:{liar_half_true_len}")

label_mapping = {
    "pants-fire": "FALSE",
    "barely-true": "FALSE",
    "half-true": "TRUE",
    "mostly-true": "TRUE"
}

LIAR['label'] = LIAR['label'].replace(label_mapping)

# print(LIAR)

True Claims:2053
False Claims:2504
Half true Claims:2627


In [4]:
POLI_FAKE = pd.read_csv("data/politifact_fake.csv")
POLI_FAKE['label'] = "FALSE"
poli_false_len = len(POLI_FAKE)
print(f"Politifact False Claims: {poli_false_len}")
POLI_FAKE.head(1)

Politifact False Claims: 432


,id,news_url,title,tweet_ids,label
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...,FALSE


In [5]:
POLI_REAL = pd.read_csv("data/politifact_real.csv")
POLI_REAL['label'] = "TRUE"

poli_true_len = len(POLI_REAL)

print(f"Politifact True Claims: {len(POLI_REAL)}")
POLI_REAL.head(1)

Politifact True Claims: 624


,id,news_url,title,tweet_ids,label
0,politifact14984,http://www.nfib-sbet.org/,National Federation of Independent Business,967132259869487105\t967164368768196609\t967215...,TRUE


In [6]:
GOSSIP_FAKE = pd.read_csv("data/gossipcop_fake.csv")
GOSSIP_FAKE['label'] = "FALSE"


gossip_false_len = len(GOSSIP_FAKE)
print(f"Gossipcop False Claims: {gossip_false_len}")
GOSSIP_FAKE.head(1)


Gossipcop False Claims: 5323


,id,news_url,title,tweet_ids,label
0,gossipcop-2493749932,www.dailymail.co.uk/tvshowbiz/article-5874213/...,Did Miley Cyrus and Liam Hemsworth secretly ge...,284329075902926848\t284332744559968256\t284335...,FALSE


In [7]:
GOSSIP_REAL = pd.read_csv("data/gossipcop_real.csv")
GOSSIP_REAL['label'] = "TRUE"

gossip_true_len = len(GOSSIP_REAL)
print(f"Gossipcop True Claims: {gossip_true_len}")
GOSSIP_REAL.head(1)


Gossipcop True Claims: 16817


,id,news_url,title,tweet_ids,label
0,gossipcop-882573,https://www.brides.com/story/teen-mom-jenelle-...,Teen Mom Star Jenelle Evans' Wedding Dress Is ...,912371411146149888\t912371528343408641\t912372...,TRUE


In [8]:
total_true_len = liar_true_len + poli_true_len + gossip_true_len
total_false_len = liar_false_len + poli_false_len + gossip_false_len

print(f"Total True claims: {total_true_len}")
print(f"Total False claims: {total_false_len}")


Total True claims: 19494
Total False claims: 8259


In [9]:
LIAR['source'] = "LIAR"
POLI_FAKE['source'] = "POLITIFACT_FAKE"
POLI_REAL['source'] = "POLITIFACT_REAL"
GOSSIP_FAKE['source'] = "GOSSIP_FAKE"
GOSSIP_REAL['source'] = "GOSSIP_REAL"

aggregated_df = pd.concat([
    LIAR[["source","title","label"]],
    POLI_FAKE[["source","title","label"]],
    POLI_REAL[["source","title","label"]],
    GOSSIP_REAL[["source","title","label"]],
    GOSSIP_FAKE[["source","title","label"]],
], ignore_index=True)

In [10]:
# aggregated_df.loc[aggregated_df['source'] == "POLITIFACT_FAKE", "title"].head(5)
aggregated_df.loc[aggregated_df['title']=="Tonight in Ohio, more people came out to vote for Barack Obama in an unopposed race than voted for (Mitt) Romney and (Rick) Santorum combined."]

,source,title,label
2560,LIAR,"Tonight in Ohio, more people came out to vote ...",FALSE


In [11]:
aggregated_df['label'].unique()

array(['TRUE', 'FALSE'], dtype=object)

In [12]:
aggregated_df.dropna(subset=['title','label'], inplace=True)

In [13]:
len(aggregated_df)

35983

In [14]:
# Drop the duplicates titles from the dataset
aggregated_df.drop_duplicates(subset=['title'], inplace=True)

In [ ]:
print(f"Length of Aggregated DF:{len(aggregated_df)}")
true_df = aggregated_df[aggregated_df['label'] == "TRUE"]

print(f"Length of TRUE DF:{len(true_df)}")
# true_df.to_csv("data/Real_df_for_RAFC.csv", index=False)
# true_df['label'].unique()


Length of Aggregated DF:34485
Length of TRUE DF:23644


In [ ]:
aggregated_df.head(10)

,source,title,label
0,LIAR,Building a wall on the U.S.-Mexico border will...,TRUE
1,LIAR,Wisconsin is on pace to double the number of l...,FALSE
2,LIAR,Says John McCain has done nothing to help the ...,FALSE
3,LIAR,Suzanne Bonamici supports a plan that will cut...,TRUE
4,LIAR,When asked by a reporter whether hes at the ce...,FALSE
5,LIAR,Over the past five years the federal governmen...,TRUE
6,LIAR,Says that Tennessee law requires that schools ...,TRUE
7,LIAR,"Says Vice President Joe Biden ""admits that the...",FALSE
8,LIAR,Donald Trump is against marriage equality. He ...,TRUE
9,LIAR,We know that more than half of Hillary Clinton...,FALSE


In [17]:
import re

def preprocess_text(text):
    """Method to convert titles into more processed form before using"""
    text = text.lower()
    text = re.sub(r'http\S+|www\S+', '', text)

    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text.strip()

aggregated_df['title'] = aggregated_df['title'].apply(preprocess_text)

In [18]:
aggregated_df.head(10)


,source,title,label
0,LIAR,building a wall on the usmexico border will ta...,TRUE
1,LIAR,wisconsin is on pace to double the number of l...,FALSE
2,LIAR,says john mccain has done nothing to help the ...,FALSE
3,LIAR,suzanne bonamici supports a plan that will cut...,TRUE
4,LIAR,when asked by a reporter whether hes at the ce...,FALSE
5,LIAR,over the past five years the federal governmen...,TRUE
6,LIAR,says that tennessee law requires that schools ...,TRUE
7,LIAR,says vice president joe biden admits that the ...,FALSE
8,LIAR,donald trump is against marriage equality he w...,TRUE
9,LIAR,we know that more than half of hillary clinton...,FALSE


In [19]:
class_distribution = aggregated_df['label'].value_counts(normalize=True)
print(class_distribution)

label
TRUE     0.685631
FALSE    0.314369
Name: proportion, dtype: float64


In [20]:
from sklearn.utils import resample

majority_class = aggregated_df[aggregated_df['label'] == "TRUE"]
minority_class = aggregated_df[aggregated_df['label'] == "FALSE"]

minority_unsampled = resample(minority_class, replace=True, n_samples=len(majority_class), random_state=42)

In [21]:
balanced_df = pd.concat([majority_class, minority_unsampled])

In [22]:
balanced_df['label'].unique()

array(['TRUE', 'FALSE'], dtype=object)

In [23]:
class_distribution = balanced_df['label'].value_counts(normalize=True)
print(class_distribution)

label
TRUE     0.5
FALSE    0.5
Name: proportion, dtype: float64


In [24]:
from sklearn.model_selection import train_test_split as tts

train_df, test_df = tts(balanced_df, test_size=0.2, random_state=42, stratify=balanced_df['label'])

In [25]:
train_df["title"].isnull().sum()

np.int64(0)

In [26]:
test_df['title'].isna().sum()

np.int64(0)

In [27]:
print(train_df['title'].apply(type).unique())  # Check data types in train_df
print(test_df['title'].apply(type).unique())   # Check data types in test_df

[<class 'str'>]
[<class 'str'>]


In [28]:
print("Empty titles in train_df:", (train_df['title'] == "").sum())
print("Empty titles in test_df:", (test_df['title'] == "").sum())

Empty titles in train_df: 5
Empty titles in test_df: 0


In [29]:
train_df['title'][train_df['title'] == ""] = "empty title"

C:\Users\Admin\AppData\Local\Temp\ipykernel_6872\834227359.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  train_df['title'][train_df['title'] == ""] = "empty title"


In [30]:
train_df['title'] = train_df['title'].str.strip()
test_df['title'] = test_df['title'].str.strip()

In [31]:
# Map 'TRUE' to 1 and 'FALSE' to 0
train_df['label'] = train_df['label'].map({'TRUE': 1, 'FALSE': 0})
test_df['label'] = test_df['label'].map({'TRUE': 1, 'FALSE': 0})

# Verify the mapping
print(train_df['label'].value_counts())
print(test_df['label'].value_counts())

label
1    18915
0    18915
Name: count, dtype: int64
label
0    4729
1    4729
Name: count, dtype: int64


In [32]:
# test_df.to_csv("data/balanced_test_data.csv", index=False)
# train_df.to_csv("data/balanced_train_data.csv", index=False)